In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML

In [110]:
# Dimensiones del tablero
anchura_superficie = 80
altura_superficie = 40

# Establecemos unos valores predeterminados para 'a' y el centro de las figuras
a_pred = altura_superficie / 2
c_circ_pred = (3 * (anchura_superficie / 4), altura_superficie / 2)
c_cuad_pred = (anchura_superficie / 4, altura_superficie / 2)

In [111]:
# Definimos el generador de números pseudoaleatorios
rng = np.random.default_rng()

In [112]:
def generar_puntos(numero_muestras, rng):
    """Genera un número determinado de puntos aleatorios.

    Esta función genera puntos con coordenadas x e y aleatorias.
    Los valores de X son numeros reales entre 0 y 80 (la anchura de la superficie).
    Los valores de Y son números reales entre 0 y 40 (la altura de la superficie).
    Finalmente, guarda todos los puntos generados en un array

    Args:
        numero_muestras (int) : Cantidad de puntos a generar.
        rng (Generator) : Generador de números pseudoaleatorios de numpy.

    Returns:
        Un array de dimensiones (numero_muestras, 2). Cada fila es una coordenada (x,y)
        de números de coma flotante
    """
    
    return np.array([(rng.uniform(0, anchura_superficie), rng.uniform(0, altura_superficie)) for n in range(numero_muestras)])

In [113]:
def dentro_circulo(coordenadas, centro = c_circ_pred, a = a_pred):
    """Determina si unas coordenadas se encuentran dentro del
    área del círculo.

    Esta función analiza las coordenadas X e Y de un punto y el
    área del círculo en base a sus dimensiones y su posición en la
    superficie del experimento.

    Args:
        coordenadas ([float, float]) : coordenadas del punto a analizar.
        centro (tuple(float, float), Default: (60.0, 20.0)): coordenadas de la posición del centro del círculo
        a (float, Default: 20.0) : tamaño del radio del círculo

    Returns:
        Valor booleano que determina si el punto se encuentra dentro
        del área del círculo
    """
    x, y = coordenadas

    x_circulo, y_circulo = centro
    distancia_a_coordenada = (x - x_circulo)**2 + (y - y_circulo)**2

    return (distancia_a_coordenada < a**2)

In [114]:
def dentro_cuadrado(coordenadas, centro = c_cuad_pred, a = a_pred):
    """Determina si unas coordenadas se encuentran dentro del
    área del cuadrado.

    Esta función analiza las coordenadas X e Y de un punto y el
    área del cuadrado en base a sus dimensiones y su posición en la
    superficie del experimento.

    Args:
        coordenadas ([float, float]) : coordenadas del punto a analizar.
        centro (tuple(float, float), Default: (20.0, 20.0)): coordenadas de la posición del centro del cuadrado
        a (float, Default: 20.0) : tamaño del lado del cuadrado

    Returns:
        Valor booleano que determina si el punto se encuentra dentro
        del área del cuadrado
    """
    x, y = coordenadas

    x_cuadrado, y_cuadrado = centro
    x_lim_inf = x_cuadrado - (a / 2)
    x_lim_sup = x_cuadrado + (a / 2)
    y_lim_inf = y_cuadrado - (a / 2)
    y_lim_sup = y_cuadrado + (a / 2)

    return (x_lim_inf < x < x_lim_sup) and (y_lim_inf < y < y_lim_sup)

In [115]:
def realizar_simulacion(puntos, c_circ = c_circ_pred, c_cuad = c_cuad_pred, a = a_pred):
    """Determina en qué figura ha caído cada uno de los puntos generados

    Esta función obtiene el resultado de comprobar en dónde ha caído cada
    uno de los puntos generados. Para cada figura, guarda un 1 si el punto
    ha caído dentro, un 0 en caso contrario.

    Args:
        puntos (ndarray(dtype=float, ndim=2)) : array de puntos generados
        c_circ (tuple(float, float), Default: (60.0, 20.0)): coordenadas de la posición del centro del círculo
        c_cuad (tuple(float, float), Default: (20.0, 20.0)): coordenadas de la posición del centro del cuadrado
        a (float, Default: 20.0) : tamaño de las figuras

    Returns:
        Array de números enteros de dos dimensiones. El primer número de cada fila (0,1) determina si el punto
        está dentro del círculo. El segundo número (0,1), si está dentro del cuadrado.
    """    
    return np.array([(int(dentro_circulo(n, c_circ, a)), int(dentro_cuadrado(n, c_cuad, a))) for n in puntos])


In [118]:
def repetir_experimento(num_muestras, num_reps, rng, c_circ = c_circ_pred, c_cuad = c_cuad_pred, a = a_pred):
    """Repite la simulación de una cantidad de puntos y guarda los resultados

    Esta función genera todos los puntos aleatorios necesarios para el experimento.
    Después, realiza tantas simulaciones como indique num_reps utilizando num_muestras
    en cada una. Finalmente, guarda todos los resultados por separado en un array.

    Args:
        num_muestras (int) : cantidad de puntos por simulación
        num_reps (int) : cantidad de veces a repetir el experimento
        rng (Generator) : generador de números pseudoaleatorios de numpy
        c_circ (tuple(float, float), Default: (60.0, 20.0)): coordenadas de la posición del centro del círculo
        c_cuad (tuple(float, float), Default: (20.0, 20.0)): coordenadas de la posición del centro del cuadrado
        a (float, Default: 20.0) : tamaño de las figuras

    Returns:
        Array de números enteros de dos dimensiones. 
        El primer número de cada fila son los puntos que han caído en el círculo en cada experimento.
        El segundo número de cada fila son los puntos que han caído en el cuadrado en cada experimento.
    """    
    # Generamos suficientes muestras para todas las repeticiones
    puntos_generados = generar_puntos(num_muestras * num_reps, rng)

    # Realizamos num_reps simulaciones, cada simulación con n_muestras
    resultados = np.array([realizar_simulacion(puntos_generados[n*num_muestras:(n+1)*num_muestras], c_circ, c_cuad, a) for n in range(num_reps)])

    # Para cada resultado del experimento, sumamos la cantidad de puntos en cada figura
    suma_puntos = np.array([[resultados[n][:,0].sum(), resultados[n][:,1].sum()] for n in range(num_reps)])

    return suma_puntos

In [125]:
def aproximar_pi_separado (num_muestras, num_reps, rng, c_circ = c_circ_pred, c_cuad = c_cuad_pred, a = a_pred):
    """Aproxima PI para cada experimento realizado

    Esta función obtiene los resultados de los experimentos por separado
    y realiza la aproximación de PI para cada uno de ellos

    Args:
        num_muestras (int) : cantidad de puntos por simulación
        num_reps (int) : cantidad de veces a repetir el experimento
        rng (Generator) : generador de números pseudoaleatorios de numpy
        c_circ (tuple(float, float), Default: (60.0, 20.0)): coordenadas de la posición del centro del círculo
        c_cuad (tuple(float, float), Default: (20.0, 20.0)): coordenadas de la posición del centro del cuadrado
        a (float, Default: 20.0) : tamaño de las figuras

    Returns:
        Array de floats que contienen la aproximación obtenida en cada experimento
    """ 
    puntos_resultado = repetir_experimento(num_muestras, num_reps, rng, c_circ, c_cuad, a)
    aprox_pi = np.zeros(num_reps)

    for n in range(num_reps): # Para cada experimento

        if puntos_resultado[n][1] == 0: # Si no han caído puntos en el cuadrado, evitamos la división entre cero
            aprox_pi[n] = 0
        else:
            aprox_pi[n] = puntos_resultado[n][0]/puntos_resultado[n][1]

    return aprox_pi


In [126]:
def aproximar_pi_junto (num_muestras, num_reps, rng, c_circ = c_circ_pred, c_cuad = c_cuad_pred, a = a_pred):
    """Aproxima PI para el total de los experimentos realizados

    Esta función obtiene el resultados de todos los experimentos
    y realiza la aproximación de PI de los experimentos en conjunto

    Args:
        num_muestras (int) : cantidad de puntos por simulación
        num_reps (int) : cantidad de veces a repetir el experimento
        rng (Generator) : generador de números pseudoaleatorios de numpy
        c_circ (tuple(float, float), Default: (60.0, 20.0)): coordenadas de la posición del centro del círculo
        c_cuad (tuple(float, float), Default: (20.0, 20.0)): coordenadas de la posición del centro del cuadrado
        a (float, Default: 20.0) : tamaño de las figuras

    Returns:
        Float con la aproximación de PI del total de experimentos realizados
    """ 
    puntos_resultado = repetir_experimento(num_muestras, num_reps, rng, c_circ, c_cuad, a)
    puntos_circulo  = puntos_resultado[:,0].sum()
    puntos_cuadrado = puntos_resultado[:,1].sum()

    if puntos_cuadrado == 0:
        return 0
    else:
        return puntos_circulo/puntos_cuadrado

In [127]:
# Generamos las coordenadas
N_muestras = int(1e4)
N_reps = 100

rng = np.random.default_rng(23)
aprox_separados = aproximar_pi_separado (N_muestras, N_reps, rng).mean()

rng = np.random.default_rng(23)
aprox_juntos = aproximar_pi_junto(N_muestras, N_reps, rng)

print(f"Las aproximaciones de Pi son {aprox_separados} (media por separado) y {aprox_juntos} (aproximación en conjunto)")

Las aproximaciones de Pi son 3.1464461209317967 (media por separado) y 3.1435450327947527 (aproximación en conjunto)
